# 卷積的概念

## 簡單卷積

In [1]:
import numpy as np

# 設定隨機種子，確保每次執行產生的亂數都一樣（方便除錯）
np.random.seed(5)

# 產生一個包含 10 個元素的陣列，數值介於 1 到 30 之間，資料型態為長整數 (long)
x = np.random.randint(low=1, high=30, size=10, dtype='l')
print(x)

# 定義權重陣列（或是過濾器 Filter/Kernel）
w = np.array([1.2, 0.3, 0.5])

# 取得原始資料 x 的長度與權重 w 的長度
n = x.size
K = w.size

# 初始化一個全為 0 的陣列 z，用來存放計算結果
# 長度設定為 n-K+1 是為了確保滑動視窗不會超出邊界 (Valid Padding)
z = np.zeros(n-K+1)

# 開始跑迴圈進行滑動視窗計算
for i in range(n-K+1):
    # 取出 x 的子部分與權重 w 進行元素對應相乘後加總 (點積運算)
    z[i] = np.sum(x[i:i+K] * w)

# 輸出權重與最終計算出來的結果
print(w)
print(z)

[ 4 15 16  7 23 17 10  9  5  8]
[1.2 0.3 0.5]
[17.3 26.3 32.8 23.8 37.7 27.9 17.2 16.3]


## Same 與 Full 卷積

In [2]:
def conv1d(x, w, pad):
    # 取得輸入資料 x 的長度與權重 w (Kernel) 的長度
    n = x.size
    K = w.size

    # 計算總共要補零的數量（左右兩邊各補 pad 個 0）
    P = 2 * pad

    # 計算輸出陣列的長度 (公式：輸入長度 + 總填充 - 核心長度 + 1)
    n_o = n + P - K + 1

    # 初始化輸出陣列 y
    y = np.zeros(n_o)

    # 判斷是否需要進行 Padding
    if P > 0:
        # 建立一個補零後的新陣列，長度為原本長度加上 P
        x_pad = np.zeros(n + P)
        # 將原本的資料 x 塞進中間，避開頭尾需要補 0 的位置
        x_pad[pad:-pad] = x
    else:
        # 如果不需補零，直接使用原始資料
        x_pad = x

    # 開始執行滑動視窗運算
    for i in range(n_o):
        # 在補零後的陣列上，取出對應區段與權重相乘並加總
        y[i] = np.sum(x_pad[i:i + K] * w)

    return y

In [3]:
# 執行 Same 卷積：透過補 1 層零 (pad=1)，讓輸出的長度儘可能跟輸入一樣
y1 = conv1d(x, w, 1)
# 印出原始資料長度、權重長度以及計算後的長度
print(x.size, w.size, y1.size)
print("same: ", y1)

# 執行 Full 卷積：補上足夠的零 (pad=2)，讓權重只要「沾到邊」就開始計算
# 這會讓輸出的長度達到最大值 (n + K - 1)
y2 = conv1d(x, w, 2)
print(x.size, w.size, y2.size)
print("full: ", y2)

10 3 10
same:  [ 8.7 17.3 26.3 32.8 23.8 37.7 27.9 17.2 16.3  8.4]
10 3 12
full:  [ 2.   8.7 17.3 26.3 32.8 23.8 37.7 27.9 17.2 16.3  8.4  9.6]
